### Еще немного про LIPM и SLIP

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy import linalg
import copy
import math

class LIP:
    def __init__(self, m, g, h, dt, x0, xd0, y0, yd0, xleg0, yleg0):
        self.m = m
        self.g = g
        self.h = h
                
        self.dt = dt
        
        self.x = np.array([[x0],
                           [xd0],
                           [y0],
                           [yd0]])
        
        self.xleg = xleg0
        self.yleg = yleg0
        
        self.A = np.array([[0,          1, 0,          0],
                           [self.g / h, 0, 0,          0],
                           [0,          0, 0,          1],
                           [0,          0, self.g / h, 0]])
        
    def get_state(self):
        return self.x, self.xleg, self.yleg
    
    def propagate_system(self):
        xd = self.A @ (self.x - np.array([[self.xleg], [0], [self.yleg], [0]]))
        
        self.x += xd * self.dt
        
    def set_leg_position(self, newx, newy):
        self.xleg = newx
        self.yleg = newy
    
def run_LIP(m, g, h, x0, xd0, y0, yd0, dt, iter_num,
            step_time, ref_x, ref_y):
    lip = LIP(m, g, h, dt, x0, xd0, y0, yd0, x0, y0)

    x_traj = []
    y_traj = []
    x_leg_traj = []
    y_leg_traj = []
    
    t = 0
    time_elapsed = 0
    
    for i in range(iter_num):
        curr_t = dt * i
        
        state, xleg, yleg = lip.get_state()
                
        if (time_elapsed >= step_time):
            time_elapsed = 0
            
            #print(state[0, 0] - ref_x[i])
            
            newx = ref_x[i] + (state[0, 0] - ref_x[i]) * 1.5 + state[1, 0] * 0.46
            newy = ref_y[i] + (state[2, 0] - ref_y[i]) * 1.5 + state[3, 0] * 0.46
            
            lip.set_leg_position(newx, newy)
            #print("new leg")
            
        x_traj.append(state[0, 0])
        y_traj.append(state[2, 0])
        x_leg_traj.append(xleg)
        y_leg_traj.append(yleg)
        
        lip.propagate_system()
        
        time_elapsed += dt
        #print(time_elapsed)
    
    return x_traj, y_traj, x_leg_traj, y_leg_traj

iter_num = 200
h = 2

# ref_x = [2 * np.cos(i / 7) for i in range(iter_num)]
# ref_y = [2 * np.sin(i / 7) for i in range(iter_num)]

ref_x, ref_y = [], []

for i in range(iter_num):
    angle1 = i / 50
    angle2 = i / 3.5
    
    x = np.cos(angle1) * 7 #(7 + 0.18 * np.cos(angle2))
    y = np.sin(angle1) * 7 #(7 + 0.18 * np.cos(angle2))
    
    ref_x.append(x)
    ref_y.append(y)

x_hist, y_hist, x_leg_traj, y_leg_traj = run_LIP(m = 1.0,
    g = 10, h = h, x0 = ref_x[0] + 0.01, xd0 = 0.004,
    y0 = ref_y[0] + 0.01, yd0 = -0.005,
    dt = 0.1, iter_num = iter_num, step_time = 0.7, ref_x = ref_x,
                                                 ref_y = ref_y)

print("simulation complete")

plt.plot(np.linspace(0, iter_num, iter_num), x_hist)
plt.plot(np.linspace(0, iter_num, iter_num), ref_x)
plt.show()

plt.plot(x_hist, y_hist)
plt.plot(ref_x, ref_y)
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
import matplotlib.animation
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

vis_step = 5

def draw_LIP(i):
    plt.cla()

    ax.plot([x_hist[i * vis_step], x_leg_traj[i * vis_step]],
            [y_hist[i * vis_step], y_leg_traj[i * vis_step]],
            [h, 0], 'bo-', label='lipm')
    
    ax.set_title("LIP walking")
    ax.set_xlabel("X (см)")
    ax.set_ylabel("Y (см)")
    ax.set_zlabel("Z (см)")
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)
    ax.set_zlim(0, 10)
    ax.legend(fontsize="x-large")
    ax.grid()

#xs = [1, 2, -3]

#for x1 in xs:
#    draw_LIP(x1)

#x = np.linspace(0,10,100)

matplotlib.animation.FuncAnimation(fig, draw_LIP, frames=40) #iter_num // vis_step)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy import linalg
import copy
import math

class Leg:
    def __init__(self, l0, k, x0, in_contact = False):
        self.l0 = l0
        self.k = k
        self.x = x0
        self.in_contact = in_contact
    
    def calc_distance_to_point(self, x, y):
        return math.sqrt((x - self.x)**2 + y**2)
    
    def update_contact_status(self, x, y):
        d = self.calc_distance_to_point(x, y)
        
        if (d > self.l0):
            self.in_contact = False
        
        else:
            self.in_contact = True
            
    def draw(self, canvas, cx, cy, x, y, scale):
        leg_color = (23, 234, 21)

        if (self.in_contact == False):
            return
            leg_color = (123, 134, 21)

        cv2.line(canvas, (int(cx + x * scale), int(cy - y * scale)),
                         (int(cx + self.x * scale), cy), leg_color, 2)

class SLIP:
    def __init__(self, m = 1.0, g = 10.0,
                 x0 = 0.2, xd0 = -0.3, y0 = 1.2, yd0 = -0.1,
                 dt = 0.001, Q = np.eye(4), R = np.eye(1),
                 WIND_X = 700, scale = 100):
        self.m = m
        self.g = g
        
        self.x = np.array([[x0], [xd0], [y0], [yd0]])
        self.dt = dt
        
        self.Q = Q
        self.R = R
        
        self.WIND_X = WIND_X
        self.scale = scale
        self.target_x = 0.0
        
        self.legs = []
        
        self.walking_phase = "standing"
        
    def get_state(self):
        return self.x
    
    def add_leg(self, new_leg):
        self.legs.append(new_leg)
    
    def propagate_system(self, u):
        x  = self.x[0, 0]
        xd = self.x[1, 0]
        y  = self.x[2, 0]
        yd = self.x[3, 0]
        
        g = self.g
        m = self.m        
        
        xdd = 0
        ydd = 0
        
        for leg in self.legs:
            if (leg.in_contact == True):
                d = leg.calc_distance_to_point(x, y)
                
                xdd += -leg.k * (x - leg.x) * (1 - leg.l0 / d)
                ydd += -leg.k * y * (1 - leg.l0 / d)
        
        xdd /= self.m
        ydd = ydd / self.m - self.g
        
        self.x[1, 0] += xdd * self.dt
        self.x[0, 0] += self.x[1, 0] * self.dt
        
        self.x[3, 0] += ydd * self.dt
        self.x[2, 0] += self.x[3, 0] * self.dt
        
        for leg in self.legs:
            leg.update_contact_status(x, y)
        
        #if
        
    def draw(self, cx, cy, scale = 100, color = (234, 123, 123), canvas = None):
        if (canvas is None):
            canvas = np.ones((700, 700, 3)) * 0
        
        h, w, _ = canvas.shape

        cv2.line(canvas, (0, h // 2), (w, h // 2), color, 2)
        
        cv2.circle(canvas, (int(cx + self.x[0, 0] * scale), int(cy - self.x[2, 0] * scale)), 17, color, 2)

        for leg in self.legs:
            leg.draw(canvas, cx, cy, self.x[0, 0], self.x[2, 0], scale)

        return canvas
    
def state_action_cost(x, u, Q, R):
    cost = x.T @ Q @ x + u.T @ R @ u
    
    return cost

def episode_cost(x_hist, u_hist, Q, R):
    total_cost = 0
    cost_hist = []
    
    for x, u in zip(x_hist, u_hist):
        cost = state_action_cost(x, u, Q, R)
        
        total_cost += cost
        cost_hist.append(cost)
    
    return total_cost, cost_hist

def run_slip_episode(m = 1.0, g = 10.0,
                 x0 = 0.2, xd0 = -0.3, y0 = 1.2, yd0 = -0.1,
                 dt = 0.01, Q = np.eye(4), R = np.eye(1),
                 WIND_X = 700, scale = 100):
    WIND_X = 700
    WIND_Y = 700
    canvas = np.ones((700, 700, 3), np.uint8) * 70
    
    slip = SLIP(m, g, x0, xd0, y0, yd0,
                dt, Q, R, WIND_X, scale)

    x_first_leg = -0.99 - 5
    step = 1.6
    legs_num = 16

    for i in range(legs_num):
        new_leg = Leg(5, 100, x_first_leg + i * step, False)
        slip.add_leg(new_leg)
    
    iter_num = 2750
    i = 0

    x_traj = []
    y_traj = []
    u_traj = []
    
    Q = np.eye(4) * 1
    
    #Q[:2, :2] *= 10
    
    R = np.eye(1)
    
    horizon = 10

    F_max = 20
    
    updating = False
    
    while(True):
        state = slip.get_state()

        control = np.array([[0]])
        
        slip.propagate_system(control)
        
        x_traj.append(state[0, 0])
        y_traj.append(state[2, 0])
        u_traj.append(control)
        
        canvas = cv2.addWeighted(canvas, 0.93, canvas, 0, 0)
        slip.draw(WIND_X // 2, WIND_Y // 2, canvas = canvas, scale = scale)

        cv2.imshow("slip", canvas)
        
        if (updating == False):
            cv2.waitKey(0)
            updating = True
        
        i += 1

        if (i > iter_num):
            break
        
        key = cv2.waitKey(10) & 0xFF
        
        if (key == ord('q')):
            break
    
    cv2.destroyAllWindows()
    cv2.waitKey(10)
    
    return x_traj, y_traj

Q = np.eye(4)
Q[0, 0] *= 10
#Q[1, 1] *= 0.1
R = np.eye(1) * 1

x_hist, y_hist = run_slip_episode(m = 1.0, g = 10,
                 x0 = -5, xd0 = 1.3, y0 = 4.9, yd0 = 0.0,
                 dt = 0.01, Q = np.eye(4), R = np.eye(1),
                 WIND_X = 700, scale = 50)

def plot_1d(data):
    plt.figure(figsize=(15, 4))
    plt.plot(data)
    plt.show()

plot_1d(x_hist)
plot_1d(y_hist)

plt.figure(figsize=(15, 15))
plt.plot(x_hist, y_hist)
plt.show()

### Symbolic algebra

Symbols, derivatives, functions

In [ ]:
import sympy as smp

t, m, g = smp.symbols('t m g')

In [ ]:
t

In [ ]:
g + m * t**2

In [ ]:
#r'' for rastering
theta = smp.symbols(r'\theta', cls = smp.Function)
theta = theta(t)

theta

In [ ]:
theta_d = smp.diff(theta, t)

theta_dd = smp.diff(theta, t, t)

In [ ]:
theta_d

In [ ]:
theta_dd

In [ ]:
x, y = smp.symbols('x y', cls = smp.Function)

x = x(theta)
y = y(theta)

In [ ]:
x

In [ ]:
x = smp.sin(theta)
y = smp.cos(theta)

y

#### Building Lagrangian

In [ ]:
T = 1 / 2 * m * (smp.diff(x, t)**2 + smp.diff(y, t)**2)

V = m * g * y

In [ ]:
T

In [ ]:
L = T - V

In [ ]:
L

#### Equations

$\dfrac{d L}{d \theta} - \dfrac{d}{d t}\dfrac{d L}{d \dot{\theta}} = 0$

In [ ]:
EL = smp.diff(L, theta) - smp.diff(L, theta_d, t)

In [ ]:
EL

In [ ]:
EL.simplify()

#### Solving for derivatives

In [ ]:
deriv_2 = smp.solve(EL, theta_dd)[0]
deriv_1 = theta_d

In [ ]:
deriv_2

#### Sanity check: let us obtain Newton's Second Law for point mass on 

- Lagrangian (kinetic enery)
- Euler-Lagrange equation
- Folving for $\ddot{x}$

In [ ]:
import numpy as np
import sympy as smp

t, m, F = smp.symbols('t, m, F')
x = smp.symbols('x', cls = smp.Function)

x = x(t)
x_d = smp.diff(x, t)
x_dd = smp.diff(x_d, t)

L = m * x_d**2 / 2

EL1 = - smp.diff(L, x) + smp.diff(L, x_d, t) - F

EL1

In [ ]:
x_dd_rhs = smp.solve(EL1, x_dd)[0]

x_dd_rhs

#### Lambdify

In [ ]:
deriv_2f = smp.lambdify((theta, theta_d, g), deriv_2)
deriv_1f = smp.lambdify((theta_d), deriv_1)

In [ ]:
deriv_2f(1, 2, 3)

In [ ]:
deriv_1f(4)

#### Let us simulate a system

$\frac{d}{dt}
\begin{pmatrix}
\theta \\
\dot{\theta}
\end{pmatrix}=
$
$\begin{pmatrix}
\dot{\theta} \\
\ddot{\theta}
\end{pmatrix} = f(x, u)
$

In [ ]:
import numpy as np
import sympy as smp

t, g, l, m = smp.symbols('t, g, l, m')

#t**2 / g

theta = smp.symbols(r'\theta', cls = smp.Function)

theta = theta(t)
theta_d = smp.diff(theta, t)
theta_dd = smp.diff(theta_d, t)

T = m / 2 * l**2 * theta_d**2

V = m * g * l * smp.cos(theta)

L = T - V

EL1 = smp.diff(L, theta) - smp.diff(L, theta_d, t).simplify()

deriv2 = smp.solve(EL1, theta_dd)

EL1.simplify()

In [ ]:
theta_dd_rhs = deriv2[0]

theta_dd_rhs

In [ ]:
theta_d_f = smp.lambdify((theta_d), theta_d)
theta_dd_f = smp.lambdify((g, theta, l), theta_dd_rhs)

In [ ]:
from scipy.integrate import odeint
import matplotlib.pyplot as plt

#print(theta_dd_f(np.pi**2, 1, 1))

g = 10
l = 1

def xdot(x, t):
    return [
        theta_d_f(x[1]),
        theta_dd_f(g, x[0], l)
    ]

t = np.linspace(0, 10, 1000)

traj = odeint(xdot, y0 = [0.05, -0.03], t = t)

In [ ]:
theta_traj = traj.T[0]

#print(theta_traj)

plt.plot(theta_traj)
plt.plot()
plt.show()

#### Let us simulate a double pendulum

In [ ]:
import numpy as np
import sympy as smp

t, g, l1, l2, m1, m2 = smp.symbols('t, g, l1, l2, m1, m2')

alpha = smp.symbols(r'\alpha', cls = smp.Function)
alpha = alpha(t)
alpha_d = smp.diff(alpha, t)
alpha_dd = smp.diff(alpha_d, t)

beta = smp.symbols(r'\beta', cls = smp.Function)
beta = beta(t)
beta_d = smp.diff(beta, t)
beta_dd = smp.diff(beta_d, t)

x1 = l1 * smp.sin(alpha)
y1 = l1 * smp.cos(alpha)

x2 = x1 + l2 * smp.sin(alpha + beta)
y2 = y1 + l2 * smp.cos(alpha + beta)

T = m1 / 2 * (smp.diff(x1, t)**2 + smp.diff(y1, t)**2) + \
    m2 / 2 * (smp.diff(x2, t)**2 + smp.diff(y2, t)**2)

V = g * (m1 * y1 + m2 * y2)

L = T - V

EL1 = smp.diff(L, alpha) - smp.diff(L, alpha_d, t).simplify()
EL2 = smp.diff(L, beta) - smp.diff(L, beta_d, t).simplify()

#deriv2 = smp.solve(EL1, theta_dd)

#EL1.simplify()

deriv2 = smp.solve([EL1, EL2], [alpha_dd, beta_dd])

deriv2

In [ ]:
alpha_dd_rhs = deriv2[alpha_dd].simplify()

alpha_dd_rhs

In [ ]:
beta_dd_rhs = deriv2[beta_dd].simplify()

beta_dd_rhs

$\bar{x} = 
\begin{pmatrix}
\alpha \\
\beta \\
\dot{\alpha} \\
\dot{\beta}
\end{pmatrix}
$

In [ ]:
alpha_d_f = smp.lambdify((alpha_d), alpha_d)
beta_d_f = smp.lambdify((beta_d), beta_d)

alpha_dd_f = smp.lambdify((g, m1, m2, l1, l2, alpha, beta,
                           alpha_d, beta_d), alpha_dd_rhs)

beta_dd_f = smp.lambdify((g, m1, m2, l1, l2, alpha, beta,
                           alpha_d, beta_d), beta_dd_rhs)

#alpha_dd_f(1, 2, 3, 4, 5, 6, 7, 8, 9)

In [ ]:
from scipy.integrate import odeint
import matplotlib.pyplot as plt

g = 10
l1 = 0.8
l2 = 1.1
m1 = 1
m2 = 0.1

def xdot(x, t):
    return [
        alpha_d_f(x[2]),
        beta_d_f(x[3]),
        alpha_dd_f(g, m1, m2, l1, l2, x[0], x[1], x[2], x[3]),
        beta_dd_f(g, m1, m2, l1, l2, x[0], x[1], x[2], x[3])
    ]

t = np.linspace(0, 10, 1000)

traj = odeint(xdot, y0 = [0.05, -0.03, 1, 2], t = t)

alpha_traj = traj.T[0]

plt.plot(alpha_traj)
plt.show()

In [ ]:
import numpy as np
import cv2

def polar_to_cartesian(l1, l2, a, b):
    x1 = l1 * np.sin(a)        
    y1 = - l1 * np.cos(a)

    x2 = x1 + l2 * np.sin(a + b)
    y2 = y1 - l2 * np.cos(a + b)
    
    return [[0.0, 0.0], [x1, y1], [x2, y2]]

WIND_X, WIND_Y = 2800, 2200
canvas = np.ones((WIND_Y, WIND_X, 3), np.uint8) * 60

s = 460
i = 0

while (i < len(traj)):
    canvas[:, :, :] = 60
    
    p0, p1, p2 = polar_to_cartesian(l1, l2, traj.T[0, i], traj.T[1, i])
    
    cv2.line(canvas, (int(p0[0] * s + WIND_X // 2), int(p0[1] * s + WIND_Y // 2)),
                     (int(p1[0] * s + WIND_X // 2), int(p1[1] * s + WIND_Y // 2)),
                     (123, 234, 23), 5)

    cv2.line(canvas, (int(p2[0] * s + WIND_X // 2), int(p2[1] * s + WIND_Y // 2)),
                     (int(p1[0] * s + WIND_X // 2), int(p1[1] * s + WIND_Y // 2)),
                     (123, 234, 23), 5)
    
    i += 3
    
    cv2.imshow("canvas", canvas)
    
    key = cv2.waitKey(50)
    
    if (key == ord('q')):
        break

cv2.destroyAllWindows()
cv2.waitKey(50)